In [1]:
!pip install kaggle


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import json
from zipfile import ZipFile

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [7]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [8]:
# setup kaggle credentials as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]

In [9]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
# unzip the dataset file
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip", "r") as zip_ref:
  zip_ref.extractall()   # unzip the dataset file

In [11]:
#Loading the dataset
data = pd.read_csv("IMDB Dataset.csv")

In [12]:
data.shape

(50000, 2)

In [13]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [14]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [15]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [16]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

C:\Users\tanur\AppData\Local\Temp\ipykernel_3952\2568826810.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [17]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [18]:
data["sentiment"].value_counts()

sentiment
1    25000
0    25000
Name: count, dtype: int64

In [19]:
# Split Dataset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
X_train_raw = train_data["review"]
Y_train = train_data["sentiment"]
X_test_raw = test_data["review"]
Y_test = test_data["sentiment"]

In [20]:
# Tokenize and Pad Sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train_raw)

X_train = pad_sequences(tokenizer.texts_to_sequences(X_train_raw), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(X_test_raw), maxlen=200)

In [21]:
print(X_train)

[[1935    1 1200 ...  205  351 3856]
 [   3 1651  595 ...   89  103    9]
 [   0    0    0 ...    2  710   62]
 ...
 [   0    0    0 ... 1641    2  603]
 [   0    0    0 ...  245  103  125]
 [   0    0    0 ...   70   73 2062]]


In [22]:
print(X_test)

[[   0    0    0 ...  995  719  155]
 [  12  162   59 ...  380    7    7]
 [   0    0    0 ...   50 1088   96]
 ...
 [   0    0    0 ...  125  200 3241]
 [   0    0    0 ... 1066    1 2305]
 [   0    0    0 ...    1  332   27]]


In [23]:
print(Y_train)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64


In [24]:
Y_train = train_data["sentiment"].values
Y_test = test_data["sentiment"].values

In [25]:
print(Y_train)

[0 0 1 ... 0 1 1]


Hybrid Model - CNN + LSTM

In [26]:
# Hybrid CNN-LSTM Model
model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))

# CNN Layer
model.add(Conv1D(filters=128, kernel_size=5, activation="relu"))
model.add(MaxPooling1D(pool_size=2))

# LSTM Layer
model.add(LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))

# Fully Connected Layers
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

C:\Users\tanur\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [27]:
# Compile Model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

TRAINING THE MODEL

In [29]:
history = model.fit(
    X_train, Y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.2
)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 114s 221ms/step - accuracy: 0.6827 - loss: 0.5522 - val_accuracy: 0.8574 - val_loss: 0.3300
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 115s 230ms/step - accuracy: 0.8918 - loss: 0.2859 - val_accuracy: 0.8850 - val_loss: 0.2859
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 123s 246ms/step - accuracy: 0.9147 - loss: 0.2307 - val_accuracy: 0.8914 - val_loss: 0.2800
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 130s 259ms/step - accuracy: 0.9389 - loss: 0.1754 - val_accuracy: 0.8711 - val_loss: 0.3114
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 123s 245ms/step - accuracy: 0.9488 - loss: 0.1466 - val_accuracy: 0.8792 - val_loss: 0.3454


In [30]:
#Evaluate the Model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 43ms/step - accuracy: 0.8844 - loss: 0.3173
Test Loss: 0.31990551948547363
Test Accuracy: 0.8838000297546387


In [31]:
#Prediction Function
def predict_sentiment(review):
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=200)
    prediction = model.predict(padded_sequence)
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

In [32]:
# Test Prediction
new_review = "This movie was an absolute masterpiece. I loved every moment of it!"
sentiment = predict_sentiment(new_review)
print(f"Review Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Review Sentiment: positive
